In [1]:
import difflib
import json
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import sys
sys.path.insert(0, '..')
import warnings
import zlib

# local
from libs.etl.data_cleaning import generate_aep_outage_dfs, generate_duke_outage_dfs, generate_iir_boilers_dfs, determine_component
from libs.inspection_analysis.utils import analyze_inspection_df, hist_inspection_df, check_thickness, get_thickness_histogram, group_critdat, get_inspection_df
from libs.interfaces.rolodex_interface import rolodex_load
from libs.reference.constants import COMPONENT_TYPE_MAP, POWER_CUST_LIST
from libs.viz.utils import add_vbar_labels, add_hbar_labels

In [2]:
plt.style.use('dark_background')
#sns.set_style('darkgrid')

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

warnings.filterwarnings('ignore')

wd = os.getcwd()
print(wd)

C:\Users\tom.fiorelli\Documents\GitHub\davos-data-project\notebooks


In [10]:
data = rolodex_load(
    "inspection", 
    links=[
    ], 
    filters=[
    ], 
    ids=[], 
    tags={
        "site": "duke-gibson-generating-station-d87383",
        "unit": "unit-1-c53151",
        "organization": "duke-7f2faf",
    }, 
    limit=100000
)

In [11]:
boiler_inspection_df = pd.json_normalize(data)

In [12]:
boiler_inspection_df

,entry.id,entry.type,entry.data.portal.id,entry.data.portal.date,entry.data.portal.demo,entry.data.portal.name,entry.data.portal.slug,entry.data.portal.notes,entry.data.portal.job_id,entry.data.portal.status,entry.data.portal.read_only,entry.data.portal.show_excel,entry.data.portal.sort_order,entry.data.portal.data_import,entry.data.portal.external_id,entry.data.portal.zero_offset,entry.data.portal.component_id,entry.data.portal.inspection_type,entry.data.portal.ndt_method_type,entry.data.portal.target_release_time,entry.data.portal.sales_robotic_service_id,entry.data.robot_types,entry.tags.job,entry.tags.site,entry.tags.unit,entry.tags.component,entry.tags.organization
0,20221004-581e1e,inspection,3931,2022-10-04,0,None,20221004-581e1e,,1291,completed,0,1,3102,0,,None,3842,wall_thickness,ut,None,None,"[TOKA 4, TOKA 3]",job-370e26,duke-gibson-generating-station-d87383,unit-1-c53151,waterwalls-217aec,duke-7f2faf
1,20221004-565f7b,inspection,3930,2022-10-04,0,None,20221004-565f7b,,1291,completed,0,1,3101,0,,None,3843,wall_thickness,ut,None,None,"[TOKA 4, TOKA 3]",job-370e26,duke-gibson-generating-station-d87383,unit-1-c53151,slopes-36f0b5,duke-7f2faf
2,20221004-26bdfa,inspection,3932,2022-10-04,0,None,20221004-26bdfa,,1291,completed,0,1,3103,0,,None,3841,coating_thickness,mi,None,None,"[TOKA 4, TOKA 3]",job-370e26,duke-gibson-generating-station-d87383,unit-1-c53151,waterwalls-719308,duke-7f2faf


In [13]:
duke_gibson_1_inspection_slugs = ["20221004-581e1e", "20221004-565f7b"]

In [14]:
waterwalls_df = get_inspection_df("20221004-581e1e")

Making Req...
...Req Done


In [17]:
slopes_df = get_inspection_df("20221004-565f7b")

Making Req...
...Req Done


In [16]:
waterwalls_df.head()

,customer_y,plot_x,x_bin,customer_x,y_bin,plot_y,thickness
0,438.0,1312.0,1312.0,1312,438.0,438.0,0.29348
1,438.0,1313.0,1313.0,1313,438.0,438.0,0.28304
2,438.0,1314.0,1314.0,1314,438.0,438.0,0.29000
3,438.0,1315.0,1315.0,1315,438.0,438.0,0.27260
4,438.0,1317.0,1317.0,1317,438.0,438.0,0.27840


In [19]:
waterwalls_df.thickness.min()

0.17399999499320984

In [21]:
waterwalls_df.shape

(10187, 7)

In [18]:
slopes_df.head()

,plot_y,plot_x,customer_x,y_bin,customer_y,x_bin,thickness
0,2.0,73.0,73,2.0,2.0,73.0,0.20532
1,2.0,74.0,74,2.0,2.0,74.0,0.19604
2,2.0,75.0,75,2.0,2.0,75.0,0.19488
3,2.0,76.0,76,2.0,2.0,76.0,0.19488
4,2.0,77.0,77,2.0,2.0,77.0,0.19604


In [20]:
slopes_df.thickness.min()

0.08584000170230865

In [22]:
slopes_df.shape

(14332, 7)

In [23]:
10187+14332

24519